## Data Wrangling

This notebook consists of the data consolidation and cleaning of the model. 
It includes a series of joins between the sellers considered for the first iteration and different variables grouped by the sellerID from CSVs

**DF Structure**

|  Name |Comments   |
|---|---|
| DFsellers1 |original sellers DF |
| DFsellers2 | Include the threshold for the target variable (jun-2021)   |
| DFsellers3| Include the orders information and dropped the sellers without orders   |
| DFsellers4| Include the target variable   |
| DFsellers5| Include referrals information   |
| DFsellers6| Include products and categories information   |
| DFsellers7| Include financial information of credits and vouchers   |

DFsellers7 ended up with 263,265 and 32 columns (3 need to be dropped based). The columns can be labeled as transactional, operational and behavioural information.




In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scipy
from scipy import stats
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

### Sellers:
- The DF had 1,723,856, but I established a treshold of sellers with creation date >= june 2021. Prior to that period there was a national strike that affected the conditions, and also, since it is a early stage company with many iterations in different areas changed the sellers' experience.
- Additional sellers were dropped, for example, the ones banned from the business model and others labeld as ex-pioneer, which had a very different earnings schema and would add noice to the model
- Additionally, non relevan columns such as business model or referred_by_id were droped
- The last seller DF has 1,598,970 sellers

In [7]:
dfsellers1 = pd.read_csv("../Data/seller.csv")

In [8]:
dfsellers1.head(5)

,id,created_at,referred_by_id,user_id,banned_elenas,ex_pioneer,business_model
0,11391541,2022-09-23 00:10:33.003499+02,NaN,11419818,False,False,DEFAULT
1,11534279,2022-11-21 00:36:52.063471+01,NaN,11562762,False,False,DEFAULT
2,11543049,2022-11-25 13:52:46.429364+01,NaN,11571595,False,False,DEFAULT
3,11554558,2022-12-01 06:30:08.778874+01,NaN,11583127,False,False,DEFAULT
4,11567548,2022-12-07 15:24:24.01911+01,NaN,11596129,False,False,DEFAULT


In [ ]:
# Originally, there are 1.723.856 sellers
dfsellers1.count()

### Creation date & threshold

In [ ]:
dfsellers1['created_at'].dtype

In [ ]:
dfsellers1.dtypes

In [ ]:
#convert the 'created_at' column from a string to a datetime object with timezone information.
dfsellers1['created_at'] = pd.to_datetime(dfsellers1['created_at'], utc=True)
#convert the datetime objects to strings in the desired day format ('YYYY-MM-DD').
dfsellers1['created_at'] = dfsellers1['created_at'].dt.strftime('%Y-%m-%d')

#Convert it back to datetime 
dfsellers1['created_at'] = pd.to_datetime(dfsellers1['created_at'])



In [ ]:
dfsellers1.head(5)
dfsellers1.dtypes

dfsellers 1: 1.723.856 sellers, created as string, 0 referred by id
dfsellers2: copy of

In [ ]:
threshold_date = pd.to_datetime('2021-06-01')
dfsellers2 = dfsellers1[dfsellers1['created_at'] >= threshold_date]

In [ ]:
dfsellers2.count()

In [ ]:
#Banned review:
dfsellers2["banned_elenas"].value_counts()



In [ ]:
#dfsellers2 = dfsellers2[dfsellers2["banned_elenas"] != True].reset_index(drop=True)
#Sanity check
dfsellers2.head()

dfsellers2.drop("banned_elenas", axis=1, inplace=True)

In [ ]:
#Deleting the banned sellers
dfsellers2[dfsellers2["banned_elenas"] == True]

In [ ]:
#Referrals elimination
dfsellers2.drop("referred_by_id", axis=1, inplace=True)


In [ ]:
#business model review:
dfsellers2["business_model"].value_counts()
print("All the sellers have the same business model, the column will be eliminated")
dfsellers2.drop("business_model", axis=1, inplace=True)

In [ ]:
dfsellers2.count()

In [ ]:
# ex pioneer review:
dfsellers2["ex_pioneer"].value_counts()
f"All the values of the ex_pioneer are the same. The column will be dropped"

dfsellers2.drop("ex_pioneer", axis=1, inplace=True)

In [ ]:
dfsellers2.head()

In [ ]:
def nan_rev(df):
     """
    Calculates and displays information about missing values in a DataFrame.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame.

    Returns:
    - percentage_nan (pandas.Series): The percentage of missing values for each column.
    - missing_values_distribution (pandas.Series): The distribution of missing values for each column.
    """
    # NaN values
    total_nan = df.isna().sum().sum()
    print(f"{df} has {total_nan} NaN values")

    percentage_nan = df.isna().sum() / df.shape[0] * 100.0
    print("The percentage of missing values for each column is:")
    print(percentage_nan)

    missing_values_distribution = df.isna().sum()
    print("Missing values distribution:")
    print(missing_values_distribution)

    return  percentage_nan, missing_values_distribution

print(nan_rev(dfsellers2))



In [ ]:
def nan_vis(df):
     """
    Visualizes missing values in a DataFrame.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame.

    Returns:
    - str: A message indicating if there are any missing values or not.
    - If there are missing values, returns a bar graph with the missing values.
    """
    
    # Visualize missing values
    if df.isna().sum().sum() == 0:
        return "There are no missing values"
    else:
        plt.figure(figsize=(10, 6))
        plt.title("Missing Values")
        ax = sns.barplot(x=df.columns, y=df.isna().sum())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        plt.xlabel("Columns")
        plt.ylabel("Missing Values Count")
        plt.show()

print(nan_vis(dfsellers2))

In [ ]:
dfsellers2.head(5)


In [ ]:
#rename id to seller_id
dfsellers2.rename(columns={'id': 'seller_id'}, inplace=True)

### DF Orders

- I droped the sellers that have no placed orders, since they are not relevant for the business question. The number of sellers for the analysis is 279059 sellers
- The added columns are aggregates of different metric for each seller. Depending of the variable relevance during the modeling stage, some variables could be desaggregated.
- There are 67.5K NaN values of average effective delivery days. These come from sellers without effective orders, but the will be treated later.
- Those sellers also had NaN values for columns such as total earnings or average earnings. Those missing values were replaced by 0


In [ ]:
dforders1 = pd.read_csv("total_orders_agg.csv")

In [ ]:
dforders1.columns

In [ ]:
def basic_info(df):
    """
    Provides basic information about a DataFrame.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame.

    Returns:
    - list: A tuple containing lists of categorical columns and numerical columns, respectively.
    """
    print(f"The shape of the df is {df.shape}")
    print(f"Number of datapoints: {df.count()}")
    print("")
    print("Data types:")
    print(df.dtypes)
    cat_cols = df.select_dtypes(include=["object"]).columns.to_list()
    num_cols = df.select_dtypes(exclude=["object"]).columns.to_list()
    return cat_cols, num_cols


print(basic_info(dforders1))


In [ ]:
dforders1.head(10).round(2)

In [ ]:
nan_rev(dforders1)

In [ ]:
dforders1[dforders1.isna().any(axis=1)]

# They don't have effective orders
def replace_nan_with_zero(df, columns):
    """
    Replaces the NaN values with 0

    Parameters:
    - df (pandas.DataFrame): The input DataFrame.
    - List columns of the DataFrame that will be removed.

    Returns:
    - df (pandas.DataFrame): The input DataFrame with the updated values
    """

    
    df[columns] = df[columns].fillna(0)
    return df

In [ ]:

dforders2 = replace_nan_with_zero(dforders1,["total_effective_earnings", "avg_effective_earnings"])


In [ ]:
dforders2.count()
#dfsellers2.count()

In [ ]:
# dfseller3: Left join between sellers (dfseller2) and orders dforders2
dfsellers3 = pd.merge(dfsellers2, dforders2, on="seller_id", how="left")

In [ ]:
dfsellers3.count()

In [ ]:
dfsellers3.head(5)

In [ ]:
dfsellers3.count()

In [ ]:
#Review if there are 0 in the total orders
dfsellers3[dfsellers3["total_orders"] == 0].value_counts()




In [ ]:
#Drop sellers without any order
dfsellers3.drop(dfsellers3[dfsellers3["total_orders"].isna()].index, inplace=True)


In [ ]:
dfsellers3.head()

### DF target variable 1st iteration

- It is defined as a binary column for the effective orders (orders with state as completed, distribution, created, dispatched, in process, initial, ordered) and order created at >= 2023-03-30

- The name of the variable is called: order_last_month

In [ ]:
dftarget_it1 = pd.read_csv("target_var_first_iteration.csv")

In [ ]:
basic_info(dftarget_it1)

In [ ]:
dftarget_it1.head(5)

In [ ]:
#Replace the num_orders to a binary number
dftarget_it1["num_orders"] = dftarget_it1["num_orders"].map(lambda x: 1 if x >= 1 else 0)


In [ ]:
#Rename the num_orders to order_last_month
dftarget_it1.rename(columns={"num_orders": "order_last_month"}, inplace=True)

In [ ]:
#Join sellers3 with target variable, and name it dfseller4
dfsellers4 = pd.merge(dfsellers3, dftarget_it1, on="seller_id", how="left")

In [ ]:
dfsellers4.head(5)

## Referrals review

- 2 relevant columns, is_referred, that will be binary and referred_by_seller_id that will change to a binary column named has_referred
- After multiple analyses, the is_reffered column does not contain matching information for the sellers. The column will be dropped later on

In [ ]:
dfreferrals = pd.read_excel("referrals.xlsx")

In [ ]:
dfreferrals.head(5)

In [ ]:
dfreferrals.count()

In [ ]:
# Referred sellers
dfreferrals3 = dfreferrals.drop(["referred_by_seller_id"], axis=1)
#Sanity check
dfreferrals3.head(5)

In [ ]:
#Join the 2 tables
dfreferrals2 = pd.merge(dfreferrals, dfsellers2, left_on="referred_by_seller_id", right_on="seller_id", how='left')


In [ ]:
dfreferrals2.head(5)

In [ ]:
#drop columns
#dfreferrals2.head(5)
dfreferrals2.drop(["seller_id_x", "seller_id_y", "is_referred", "created_at", "user_id"], axis=1, inplace=True)
#Sanity check
dfreferrals2.head(5)

In [ ]:
#Binary column of has_referred
dfreferrals2["has_referred"] = dfreferrals2["referred_by_seller_id"].map(lambda x: 1 if x >= 1 else 0)

#Rename
dfreferrals2.rename(columns={"referred_by_seller_id": "seller_id"}, inplace=True)

#Sanity check
dfreferrals2.head()


In [ ]:
#Join between sellers and referred sellers (dfreferrals3)
dfsellers5 = pd.merge(dfsellers4, dfreferrals3, on="seller_id", how="left")

Validation that apparently there are no referrals:

- Multiple analyses were performed, and there are no matches. Apparently the selected types of both tables are different, since the created at values are quite different

In [ ]:
set(dfsellers4.seller_id).intersection(set(dfreferrals3.seller_id))

In [ ]:
dfsellers4.seller_id

In [ ]:
dfreferrals3.seller_id

In [ ]:
#One value validation
dfsellers5[dfsellers5["seller_id"] == 10030090].value_counts()

In [ ]:
#Validation of all the rows
# 0. Create a new validation df
df_referral_val = dfsellers5[["seller_id"]].copy()

# 1. Create a list of each seller_id of dfreferrals3
referral_validation = dfreferrals3["seller_id"].tolist()

# 2. Iterate through each element of dfsellers5, returning a True or False
df_referral_val["is_duplicated"] = dfsellers5["seller_id"].apply(lambda x: True if x in referral_validation else False)

# Count the values
value_counts = df_referral_val["is_duplicated"].value_counts()
print(value_counts)


In [ ]:
dfsellers5["is_referred"].sum()

In [ ]:
#Join between sellers and sellers that have referred
dfsellers5 = pd.merge(dfsellers5,dfreferrals2, on="seller_id", how="left" )

In [ ]:
dfsellers5.head()

In [ ]:
replace_nan_with_zero(dfsellers5, ["is_referred", "has_referred"])

In [ ]:
dfsellers5["has_referred"].sum()

## Products

- Include information on how the seller interacted with the different products and categories

In [ ]:
dfproducts = pd.read_csv("total_cart_agg.csv")

In [ ]:
basic_info(dfproducts)

In [ ]:
#Join with the sellers Table
dfsellers6 = pd.merge(dfsellers5, dfproducts, on="seller_id", how="left")

In [ ]:
dfsellers6.head(5)

nan_vis(dfsellers6)

In [ ]:
dfproducts2 = pd.read_csv("products_shared.csv")

In [ ]:
basic_info(dfproducts2)

dfproducts2.rename(columns={'id': 'seller_id'}, inplace=True)

dfproducts2.head(5)


In [ ]:
#Merge products shared with sellers
dfsellers6 = pd.merge(dfsellers6, dfproducts2, on="seller_id", how="left")

## Transactional information: credits & vouchers

In [ ]:
dftransactional = pd.read_csv("seller_credits.csv")

In [ ]:
basic_info(dftransactional)
nan_vis(dftransactional)

In [ ]:
replace_nan_with_zero(dftransactional,["avg_credit_amount"])

In [ ]:
dfsellers7 = pd.merge(dfsellers6, dftransactional, on="seller_id", how="left")

**vouchers**

In [ ]:
dftransactional2 = pd.read_csv("seller_vouchers.csv")

In [ ]:
basic_info(dftransactional2)
nan_vis(dftransactional2)

In [ ]:
# join sellers7 with the vouchers information
dfsellers7 = pd.merge(dfsellers7, dftransactional2, on="seller_id", how="left")

*checkpoint*

In [ ]:
#Checkpoint
dfsellers7.to_csv('elenas-it1-preprocessed2.csv')




In [ ]:
dfsellers7 = pd.read_csv('elenas-it1-preprocessed2.csv')

## Aggregate data exploration ##

In [ ]:
dfsellers7.shape

In [ ]:
def num_vis(df, row, col, colorp):
    # Initialization with the rows and columns for the subplot
    fig = make_subplots(rows=row, cols=col)
    
    # List of the numerical columns
    numeric_columns = df.select_dtypes(exclude=["object"]).columns.to_list()
    
    # Loop, iterating on each column
    for i, column in enumerate(numeric_columns):
        # Number of rows based on index
        row_num = (i // col) + 1
        # Number of columns based on index, + 1 to start on the first position and have an integer
        col_num = (i % col) + 1
        
        # Histogram of the variables
        hist = px.histogram(df, x=column, color_discrete_sequence=px.colors.qualitative.Plotly)
        
        fig.add_trace(hist.data[0], row=row_num, col=col_num)
        fig.update_xaxes(title_text=column, row=row_num, col=col_num)
        fig.update_yaxes(title_text="Count", row=row_num, col=col_num)
    
    fig.update_layout(height=400*row, width=400*col)
    fig.show()

num_vis(dfsellers7.round(1), 6, 6, "RdPu")


In [ ]:
dfsellers7.describe().transpose()


In [ ]:
#Operational metrics
def boxplot_vis(df, col1=0, col2=0, col3=0, col4=0, col5=0, col6=0):
    # Create subplot with 2 rows and 3 columns
    fig = make_subplots(rows=2, cols=3)

    # Visualization for each value
    if col1 != 0:
        fig.add_trace(px.box(data_frame=df.round(0), y=col1).data[0], row=1, col=1)
        fig.update_xaxes(title_text=col1, row=1, col=1)
        fig.update_yaxes(title_text=input("write the y axis for the plot 1: "), row=1, col=1)
    if col2 != 0:
        fig.add_trace(px.box(data_frame=df.round(0), y=col2).data[0], row=1, col=2)
        fig.update_xaxes(title_text=col2, row=1, col=2)
        fig.update_yaxes(title_text=input("write the y axis for the plot 2: "), row=1, col=2)
    if col3 != 0:
        fig.add_trace(px.box(data_frame=df.round(0), y=col3).data[0], row=1, col=3)
        fig.update_xaxes(title_text=col3, row=1, col=3)
        fig.update_yaxes(title_text=input("write the y axis for the plot 3: "), row=1, col=3)
    if col4 != 0:
        fig.add_trace(px.box(data_frame=df.round(0), y=col4).data[0], row=2, col=1)
        fig.update_xaxes(title_text=col4, row=2, col=1)
        fig.update_yaxes(title_text=input("write the y axis for the plot 4: "), row=2, col=1)
    if col5 != 0:
        fig.add_trace(px.box(data_frame=df.round(0), y=col5).data[0], row=2, col=2)
        fig.update_xaxes(title_text=col5, row=2, col=2)
        fig.update_yaxes(title_text=input("write the y axis for the plot 5: "), row=2, col=2)
    if col6 != 0:
        fig.add_trace(px.box(data_frame=df.round(0), y=col6).data[0], row=2, col=3)
        fig.update_xaxes(title_text=col6, row=2, col=3)
        fig.update_yaxes(title_text=input("write the y axis for the plot 6: "), row=2, col=3)

    # Show the figure
    fig.show()


### Average effective delivery days review:
- The DF has over 60K missing average effective delivery days, these numbers are from sellers that don't have an effective order. So the order status is either cancelled, returned, RTO or lost.

- The data of the avg_effective_delivery_days is skewed to the left, with a mean of 5.5 days and a median of 5 days. Since both results are of effective orders, I will use a higher value to fill the NaN, assuming that due to the order state, the experience was worse. 

- The NaNs were filled with 6, which is the Q3 of the values

- Additionally, there are negative values, they will be updated with the mean




In [ ]:
nan_vis(dfsellers7)

In [ ]:
# avg delivery days top occurrences
top_del_days = dfsellers7["avg_effective_delivery_days"].round(0).value_counts().sort_values(ascending=False).iloc[:10]

fig = px.bar(top_del_days, x='avg_effective_delivery_days')
fig.show()


In [ ]:
top_del_days = dfsellers7["avg_effective_delivery_days"].round(0).value_counts().sort_values(ascending=False).iloc[:10]
top_del_days = top_del_days.reset_index()  # Reset the index to make 'index' as a column

fig = px.bar(top_del_days,
              x='index',
              y='avg_effective_delivery_days',
              title="First 10 days delivery distribution")
fig.update_layout(xaxis_title="Average Delivery Days", yaxis_title="Number of orders")
fig.show()


In [ ]:
dfsellers7

In [ ]:
avg_delivery_days = dfsellers7["avg_effective_delivery_days"].round(0)

avg_delivery_days

In [ ]:
# Avg Delivery days value distribution



fig = px.histogram(avg_delivery_days,
                   x="avg_effective_delivery_days",
                   title="Avg effective delivery days distribution",
                   nbins=20)

# Calculate the mean
mean_delivery_days = np.mean(avg_delivery_days)

# Add mean line to the plot
fig.add_vline(x=mean_delivery_days, line_dash="dash", line_color="red", annotation_text=f"Mean: {mean_delivery_days:.2f}")
fig.update_layout(annotations=[dict(x=mean_delivery_days, y=1, text=f"Mean: {mean_delivery_days:.2f}",
                                    showarrow=True, arrowhead=1, ax=20, ay=-20)])

fig.show()


In [ ]:
# Avg Delivery days boxplot

_ = dfsellers7["avg_effective_delivery_days"].round(0)

fig = px.box(_,
                   x="avg_effective_delivery_days",
                   title="Avg effective delivery days distribution")

fig.show()

In [ ]:
#The avg delivery days will be changed to 6 for the nan values

dfsellers7["avg_effective_delivery_days"] = dfsellers7["avg_effective_delivery_days"].fillna(6)


In [ ]:
dfsellers7.T.info()

In [ ]:
#Sanity check:
nan_vis(dfsellers7)

In [ ]:
# Negative values - replaced with the mean
dfsellers7["avg_effective_delivery_days"] = dfsellers7["avg_effective_delivery_days"].map(lambda x: 5.45 if x < 1 else x)



In [ ]:
# Avg Delivery days boxplot

_ = dfsellers7["avg_effective_delivery_days"].round(0)

fig = px.box(_,
                   x="avg_effective_delivery_days",
                   title="Avg effective delivery days updated distribution")

fig.show()

In [ ]:
# Drop repeated column:
dfsellers7.drop("number_of_vouchers_y", axis=1, inplace=True)


Correlations

In [ ]:
#Checkpoint
dfsellers7.to_csv('elenas-it1-preprocessed3.csv', index_col=0)



In [ ]:
dfsellers8 = pd.read_csv('elenas-it1-preprocessed2.csv')